In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gymnasium as gym
from torch.distributions import Normal
env = gym.make('InvertedPendulum-v4')

import pickle
from itertools import product

import pandas as pd


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
import pickle
from itertools import product

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=64):
        super(ActorCritic, self).__init__()
        self.actor = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, action_dim)
        )
        self.critic = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )
        self.log_std = nn.Parameter(torch.zeros(action_dim))

    def forward(self, x):
        mu = self.actor(x)
        std = self.log_std.exp().expand_as(mu)
        
        value = self.critic(x)
        return mu, std, value

class PPOAgent:
    def __init__(self, input_dim, action_dim, lr=3e-4, gamma=0.99, epsilon=0.2, k_epochs=10, minibatch_size=64, gae_lambda=0.95):
        self.gamma = gamma
        self.epsilon = epsilon
        self.k_epochs = k_epochs
        self.minibatch_size = minibatch_size
        self.gae_lambda = gae_lambda
        self.actor_critic = ActorCritic(input_dim, action_dim)
        self.optimizer = optim.Adam(self.actor_critic.parameters(), lr=lr)
        self.mse_loss = nn.MSELoss()

    def select_action(self, state):
        state = torch.from_numpy(state).float()
        mu, std, _ = self.actor_critic(state)
        distribution = torch.distributions.Normal(mu, std)
        action = distribution.sample()
        log_prob = distribution.log_prob(action).sum(dim=-1)
        return action.detach().numpy(), log_prob

    def compute_gae(self, rewards, masks, values, next_value):
        gae = 0
        returns = []
        values = values + [next_value]
        for step in reversed(range(len(rewards))):
            delta = rewards[step] + self.gamma * values[step + 1] * masks[step] - values[step]
            gae = delta + self.gamma * self.gae_lambda * masks[step] * gae
            returns.insert(0, gae + values[step])
        return returns

    def update(self, trajectory):
        states, actions, log_probs_old, returns, advantages = trajectory

        log_probs_old = torch.stack(log_probs_old).detach()
        states = torch.stack(states).detach()
        actions = torch.tensor(actions).detach()
        returns = torch.tensor(returns).unsqueeze(-1).detach()
        advantages = torch.tensor(advantages)

        for _ in range(self.k_epochs):
            indices = np.arange(states.shape[0])
            np.random.shuffle(indices)
            for start in range(0, states.shape[0], self.minibatch_size):
                end = start + self.minibatch_size
                minibatch_indices = indices[start:end]
                
                minibatch_states = states[minibatch_indices]
                minibatch_actions = actions[minibatch_indices]
                minibatch_log_probs_old = log_probs_old[minibatch_indices]
                minibatch_returns = returns[minibatch_indices]
                minibatch_advantages = advantages[minibatch_indices]

                mu, std, values = self.actor_critic(minibatch_states)
                dist = torch.distributions.Normal(mu, std)
                log_probs_new = dist.log_prob(minibatch_actions).sum(dim=-1)
                entropy = dist.entropy().mean()

                ratios = torch.exp(log_probs_new - minibatch_log_probs_old)
                surr1 = ratios * minibatch_advantages
                surr2 = torch.clamp(ratios, 1 - self.epsilon, 1 + self.epsilon) * minibatch_advantages

                actor_loss = -torch.min(surr1, surr2).mean() - 0.01 * entropy
                critic_loss = self.mse_loss(values, minibatch_returns)

                loss = actor_loss + 0.5 * critic_loss

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

    def train(self, env, total_episodes=1000, horizon=2048, batch_size=64, reward_save_path='./rewards/rewards_2.pkl'):
        all_rewards = []

        states, actions, rewards, log_probs, values, masks = [], [], [], [], [], []

        for episode in range(total_episodes):
            state = env.reset()[0]
            episode_rewards = 0

            for _ in range(1000):
                action, log_prob = self.select_action(state)
                value = self.actor_critic(torch.from_numpy(state).float())[-1].item()
                next_state, reward, done, _, _ = env.step(action)

                states.append(torch.from_numpy(state).float())
                actions.append(action)
                rewards.append(reward)
                log_probs.append(log_prob)
                values.append(value)
                masks.append(1 - done)

                state = next_state
                episode_rewards += reward

                if len(states) % batch_size == 0:
                    next_value = self.actor_critic(torch.from_numpy(state).float())[-1].item()
                    returns = self.compute_gae(rewards, masks, values, next_value)
                    advantages = [ret - val for ret, val in zip(returns, values)]
                    trajectory = (states, actions, log_probs, returns, advantages)
                    self.update(trajectory)

                    states, actions, rewards, log_probs, values, masks = [], [], [], [], [], []

                if done:
                    break

            all_rewards.append(episode_rewards)
            print(f"Episode {episode + 1}, Total Reward = {episode_rewards}")

            # Save rewards after each episode
            with open(reward_save_path, 'wb') as f:
                pickle.dump(all_rewards, f)

        return all_rewards

    def evaluate(self, env, num_episodes=100):
        print("Evaluating")
        total_rewards = 0
        for _ in range(num_episodes):
            state = env.reset()
            done = False
            episode_reward = 0

            for _ in range(100):
                state = state[0] if isinstance(state, tuple) else state
                
                state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Convert state array to tensor
               
                mu, std, value = self.actor_critic(state_tensor)
                dist = torch.distributions.Normal(mu, std)
                action = dist.sample()
               
                # Ensure action remains a 1-dimensional array with a single element
                action_numpy = action.squeeze().detach().numpy()  # Squeeze to potentially reduce dimensions
                if action_numpy.ndim == 0:  # If the result is a scalar, convert it back to an array
                    action_numpy = np.array([action_numpy])
                
                next_state, reward, done, _, _ = env.step(action_numpy)
                state = next_state
                episode_reward += reward

            total_rewards += episode_reward

        average_reward = total_rewards / num_episodes
        return average_reward



In [ ]:

def run_experiments(env, param_grid):
    results = []

    for params in param_grid:
        lr, epsilon, k_epochs, batch_size = params
        print(f"Running experiment with lr={lr}, epsilon={epsilon}, k_epochs={k_epochs}, batch_size={batch_size}")

        ppo = PPOAgent(input_dim=4, action_dim=1, lr=lr, epsilon=epsilon, k_epochs=k_epochs, minibatch_size=batch_size)
        rewards = ppo.train(env, total_episodes=1000)  # Use 1000 episodes for quicker experimentation
        avg_reward = ppo.evaluate(env)
        
        result = {
            'lr': lr,
            'epsilon': epsilon,
            'k_epochs': k_epochs,
            'batch_size': batch_size,
            'average_reward': avg_reward,
            'rewards': rewards
        }
        results.append(result)

    # Convert results to DataFrame
    df = pd.DataFrame(results)

    # Save DataFrame to a CSV file
    csv_path = './rewards/ex_results_2.csv'
    df.to_csv(csv_path, index=False)
    print(f"Results saved to {csv_path}")

    return df


param_grid = list(product(
    [3e-4, 1e-3],  # Learning rates
    [0.2],     # Epsilon values
    [5, 10],         # Epochs
    [32, 64]        # Batch sizes
))

df_results_1 = run_experiments(env, param_grid)

In [32]:
df_results = pd.read_csv('./rewards/ex_results_1.csv')
df_results

,lr,epsilon,k_epochs,batch_size,average_reward,rewards
0,0.0003,0.2,5,32,100.0,"[15.0, 7.0, 6.0, 9.0, 6.0, 6.0, 6.0, 5.0, 4.0,..."
1,0.0003,0.2,5,64,100.0,"[11.0, 13.0, 7.0, 6.0, 8.0, 8.0, 5.0, 6.0, 8.0..."
2,0.0003,0.2,10,32,100.0,"[4.0, 7.0, 6.0, 14.0, 5.0, 7.0, 5.0, 10.0, 6.0..."
3,0.0003,0.2,10,64,100.0,"[15.0, 20.0, 7.0, 12.0, 7.0, 7.0, 14.0, 21.0, ..."
4,0.0010,0.2,5,32,100.0,"[7.0, 9.0, 5.0, 8.0, 7.0, 9.0, 26.0, 5.0, 7.0,..."
5,0.0010,0.2,5,64,100.0,"[7.0, 6.0, 6.0, 13.0, 7.0, 8.0, 8.0, 9.0, 5.0,..."
6,0.0010,0.2,10,32,100.0,"[17.0, 17.0, 7.0, 5.0, 8.0, 6.0, 5.0, 7.0, 4.0..."
7,0.0010,0.2,10,64,100.0,"[8.0, 4.0, 8.0, 4.0, 7.0, 8.0, 7.0, 19.0, 5.0,..."


# Tial 1: it goes up to 1000 reward really fast

In [7]:
# class ActorCritic(nn.Module):
#     def __init__(self, state_dim, action_dim, hidden_size=64):
#         super(ActorCritic, self).__init__()
#         self.actor = nn.Sequential(
#             nn.Linear(state_dim, hidden_size),
#             nn.Tanh(),
#             nn.Linear(hidden_size, hidden_size),
#             nn.Tanh(),
#             nn.Linear(hidden_size, action_dim)
#         )
#         self.critic = nn.Sequential(
#             nn.Linear(state_dim, hidden_size),
#             nn.Tanh(),
#             nn.Linear(hidden_size, hidden_size),
#             nn.Tanh(),
#             nn.Linear(hidden_size, 1)
#         )
#         self.log_std = nn.Parameter(torch.zeros(action_dim))

#     def forward(self, x):
#         mu = self.actor(x)
#         std = self.log_std.exp().expand_as(mu)
        
#         value = self.critic(x)
#         return mu, std, value

# class PPOAgent:
#     def __init__(self, input_dim, action_dim, lr=3e-4, gamma=0.99, epsilon=0.2, k_epochs=10, minibatch_size=64, gae_lambda=0.95):
#         self.gamma = gamma
#         self.epsilon = epsilon
#         self.k_epochs = k_epochs
#         self.minibatch_size = minibatch_size
#         self.gae_lambda = gae_lambda
#         self.actor_critic = ActorCritic(input_dim, action_dim)
#         self.optimizer = optim.Adam(self.actor_critic.parameters(), lr=lr)
#         self.mse_loss = nn.MSELoss()

#     def select_action(self, state):
#         state = torch.from_numpy(state).float()
#         mu, std, _ = self.actor_critic(state)
#         distribution = torch.distributions.Normal(mu, std)
#         action = distribution.sample()
#         log_prob = distribution.log_prob(action).sum(dim=-1)
#         return action.detach().numpy(), log_prob

#     def compute_gae(self, rewards, masks, values, next_value):
#         gae = 0
#         returns = []
#         values = values + [next_value]
#         for step in reversed(range(len(rewards))):
#             delta = rewards[step] + self.gamma * values[step + 1] * masks[step] - values[step]
#             gae = delta + self.gamma * self.gae_lambda * masks[step] * gae
#             returns.insert(0, gae + values[step])
#         return returns

#     def update(self, trajectory):
#         states, actions, log_probs_old, returns, advantages = trajectory

#         log_probs_old = torch.stack(log_probs_old).detach()
#         states = torch.stack(states).detach()
#         actions = torch.tensor(actions).detach()
#         returns = torch.tensor(returns).unsqueeze(-1).detach()
#         advantages = torch.tensor(advantages)

#         for _ in range(self.k_epochs):
#             indices = np.arange(states.shape[0])
#             np.random.shuffle(indices)
#             for start in range(0, states.shape[0], self.minibatch_size):
#                 end = start + self.minibatch_size
#                 minibatch_indices = indices[start:end]
                
#                 minibatch_states = states[minibatch_indices]
#                 minibatch_actions = actions[minibatch_indices]
#                 minibatch_log_probs_old = log_probs_old[minibatch_indices]
#                 minibatch_returns = returns[minibatch_indices]
#                 minibatch_advantages = advantages[minibatch_indices]

#                 mu, std, values = self.actor_critic(minibatch_states)
#                 dist = torch.distributions.Normal(mu, std)
#                 log_probs_new = dist.log_prob(minibatch_actions).sum(dim=-1)
#                 entropy = dist.entropy().mean()

#                 ratios = torch.exp(log_probs_new - minibatch_log_probs_old)
#                 surr1 = ratios * minibatch_advantages
#                 surr2 = torch.clamp(ratios, 1 - self.epsilon, 1 + self.epsilon) * minibatch_advantages

#                 actor_loss = -torch.min(surr1, surr2).mean() - 0.01 * entropy
#                 critic_loss = self.mse_loss(values, minibatch_returns)

#                 loss = actor_loss + 0.5 * critic_loss

#                 self.optimizer.zero_grad()
#                 loss.backward()
#                 self.optimizer.step()

#     def train(self, env, total_episodes=1000000, horizon=2048, batch_size=64):
#         all_rewards = []

#         states, actions, rewards, log_probs, values, masks = [], [], [], [], [], []

#         for episode in range(total_episodes):
#             state = env.reset()[0]
#             episode_rewards = 0

#             for _ in range(1000):
#                 action, log_prob = self.select_action(state)
#                 value = self.actor_critic(torch.from_numpy(state).float())[-1].item()
#                 next_state, reward, done, _, _ = env.step(action)

#                 states.append(torch.from_numpy(state).float())
#                 actions.append(action)
#                 rewards.append(reward)
#                 log_probs.append(log_prob)
#                 values.append(value)
#                 masks.append(1 - done)

#                 state = next_state
#                 episode_rewards += reward
                

#                 if len(states) % batch_size == 0:
#                     next_value = self.actor_critic(torch.from_numpy(state).float())[-1].item()
#                     returns = self.compute_gae(rewards, masks, values, next_value)
#                     advantages = [ret - val for ret, val in zip(returns, values)]
#                     trajectory = (states, actions, log_probs, returns, advantages)
#                     self.update(trajectory)

#                     states, actions, rewards, log_probs, values, masks = [], [], [], [], [], []

#                 if done:
#                     break

#             all_rewards.append(episode_rewards)
#             print(f"Episode {episode + 1}, Total Reward = {episode_rewards}")


#         return all_rewards
    
#     def evaluate(self, env, num_episodes=1000):
#         total_rewards = 0
#         for _ in range(num_episodes):
#             state = env.reset()
#             done = False
#             episode_reward = 0

#             while not done:
#                 state = state[0] if isinstance(state, tuple) else state
                
#                 state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Convert state array to tensor
               
#                 mu, std, values = self.actor_critic(state_tensor)
#                 dist = torch.distributions.Normal(mu, std)
#                 action = dist.sample()
               
#                 # Ensure action remains a 1-dimensional array with a single element
#                 action_numpy = action.squeeze().detach().numpy()  # Squeeze to potentially reduce dimensions
#                 if action_numpy.ndim == 0:  # If the result is a scalar, convert it back to an array
#                     action_numpy = np.array([action_numpy])
                
#                 next_state, reward, done, _, _ = env.step(action_numpy)
#                 state = next_state
#                 episode_reward += reward

#             total_rewards += episode_reward

#         average_reward = total_rewards / num_episodes
#         return average_reward

# # Initialize PPOAgent and train
# ppo = PPOAgent(4, 1)
# env = gym.make('InvertedPendulum-v4')
# #rewards = ppo.train(env, total_episodes=10000, horizon=2048, batch_size=64)




In [11]:
# def run_experiments(env, param_grid):
#     results = []

#     for params in param_grid:
#         print(f"Running experiment with parameters: {params+1}")
#         lr, epsilon, k_epochs, batch_size = params
#         print(f"Running experiment with lr={lr}, epsilon={epsilon}, k_epochs={k_epochs}, batch_size={batch_size}")

#         ppo = PPOAgent(input_dim=4, action_dim=1, lr=lr, epsilon=epsilon, k_epochs=k_epochs, minibatch_size=batch_size)
#         rewards = ppo.train(env, total_episodes=1000)  # Reduce the number of episodes for quicker experimentation
#         avg_reward = ppo.evaluate(env)
        
#         result = {
#             'lr': lr,
#             'epsilon': epsilon,
#             'k_epochs': k_epochs,
#             'batch_size': batch_size,
#             'average_reward': avg_reward,
#             'rewards': rewards
#         }
#         results.append(result)

#         with open('./rewards/experiment_results_trial_1.pkl', 'wb') as f:
#             pickle.dump(results, f)

#     return results

# Define the parameter grid
# param_grid = list(product(
#     [3e-4, 1e-3],  # Learning rates
#     [0.1, 0.2, 0.3],     # Epsilon values
#     [5, 10, 20],         # Epochs
#     [32, 64]        # Batch sizes
# ))



TypeError: can only concatenate tuple (not "int") to tuple

# Trial 2: might be more correct, changed, so it saves total reward per epsiode:

Running experiment with lr=0.0003, epsilon=0.2, k_epochs=5, batch_size=32
Episode 1, Total Reward = 6.0
Episode 2, Total Reward = 6.0
Episode 3, Total Reward = 4.0
Episode 4, Total Reward = 4.0
Episode 5, Total Reward = 10.0
Episode 6, Total Reward = 11.0
Episode 7, Total Reward = 7.0
Episode 8, Total Reward = 5.0
Episode 9, Total Reward = 3.0
Episode 10, Total Reward = 7.0
Episode 11, Total Reward = 19.0
Episode 12, Total Reward = 9.0
Episode 13, Total Reward = 10.0
Episode 14, Total Reward = 11.0
Episode 15, Total Reward = 8.0
Episode 16, Total Reward = 6.0
Episode 17, Total Reward = 5.0
Episode 18, Total Reward = 5.0
Episode 19, Total Reward = 13.0
Episode 20, Total Reward = 5.0
Episode 21, Total Reward = 7.0
Episode 22, Total Reward = 7.0
Episode 23, Total Reward = 8.0
Episode 24, Total Reward = 12.0
Episode 25, Total Reward = 4.0
Episode 26, Total Reward = 26.0
Episode 27, Total Reward = 6.0
Episode 28, Total Reward = 15.0
Episode 29, Total Reward = 8.0
Episode 30, Total Reward = 

KeyboardInterrupt: 

In [ ]:
#in results 1 i ran all in range 100


#in results 2 i ran in range 1000 for train and 100 for eval

,lr,epsilon,k_epochs,batch_size,average_reward,rewards
0,0.0001,0.1,5,32,10.0,"[6.0, 10.0, 4.0, 8.0, 9.0, 5.0, 7.0, 6.0, 10.0..."
1,0.0001,0.1,5,64,10.0,"[8.0, 10.0, 10.0, 6.0, 10.0, 10.0, 6.0, 10.0, ..."
2,0.0001,0.1,5,128,10.0,"[6.0, 6.0, 10.0, 4.0, 9.0, 4.0, 4.0, 9.0, 8.0,..."
3,0.0001,0.1,10,32,10.0,"[3.0, 10.0, 5.0, 4.0, 10.0, 4.0, 10.0, 6.0, 10..."
4,0.0001,0.1,10,64,10.0,"[6.0, 4.0, 10.0, 5.0, 6.0, 4.0, 5.0, 6.0, 6.0,..."
...,...,...,...,...,...,...
76,0.0010,0.3,10,64,10.0,"[10.0, 10.0, 10.0, 5.0, 8.0, 7.0, 5.0, 6.0, 4...."
77,0.0010,0.3,10,128,10.0,"[7.0, 10.0, 10.0, 8.0, 8.0, 5.0, 10.0, 4.0, 9...."
78,0.0010,0.3,20,32,10.0,"[6.0, 8.0, 9.0, 5.0, 3.0, 8.0, 10.0, 4.0, 8.0,..."
79,0.0010,0.3,20,64,10.0,"[10.0, 10.0, 5.0, 6.0, 9.0, 5.0, 10.0, 9.0, 7...."


In [ ]:
import matplotlib.pyplot as plt
import pickle
import os
from itertools import product

def plot_rewards_1(param_grid, file_template='rewards_lr_{lr}_epsilon_{epsilon}_k_epochs_{k_epochs}_batch_size_{batch_size}.pkl', interval=10000):
    plt.figure(figsize=(10, 6))

    for params in param_grid:
        lr, epsilon, k_epochs, batch_size = params
        file_name = "./rewards/"+file_template.format(lr=lr, epsilon=epsilon, k_epochs=k_epochs, batch_size=batch_size)
        
        if os.path.exists(file_name):
            with open(file_name, 'rb') as f:
                rewards = pickle.load(f)
            
            # Calculate average rewards over intervals
            avg_rewards = [sum(rewards[i:i + interval]) / interval for i in range(0, len(rewards), interval)]
            
            label = f'LR={lr}, ε={epsilon}, Epochs={k_epochs}, Batch={batch_size}'
            plt.plot(range(0, len(rewards), interval), avg_rewards, label=label)
        else:
            print(f"File {file_name} not found. Skipping...")

    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title('Total Rewards vs Episodes for Different Hyperparameters')
    plt.legend()
    plt.grid(True)
    plt.show()

# Condensed parameter grid
param_grid = list(product(
    [1e-4, 1e-3],       # Learning rates
    [0.1, 0.3],         # Epsilon values
    [5, 20],            # Epochs
    [32, 128]           # Batch sizes
))

# Plot the rewards
plot_rewards_1(param_grid)


In [ ]:
import matplotlib.pyplot as plt
import pickle
import os
from itertools import product

def plot_rewards(param_grid, file_template='rewards_lr_{lr}_epsilon_{epsilon}_k_epochs_{k_epochs}_batch_size_{batch_size}.pkl'):
    num_params = len(param_grid)
    num_cols = 4
    num_rows = (num_params + num_cols - 1) // num_cols  # Calculate rows needed

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 5 * num_rows), sharex=True, sharey=True)
    axs = axs.flatten()

    for idx, params in enumerate(param_grid):
        lr, epsilon, k_epochs, batch_size = params
        file_name = "./rewards/" + file_template.format(lr=lr, epsilon=epsilon, k_epochs=k_epochs, batch_size=batch_size)
        
        if os.path.exists(file_name):
            with open(file_name, 'rb') as f:
                rewards = pickle.load(f)
            
            label = f'LR={lr}, ε={epsilon}, Epochs={k_epochs}, Batch={batch_size}'
            axs[idx].plot(rewards, label=label)
            axs[idx].set_title(label)
            axs[idx].legend(loc='best')
        else:
            axs[idx].set_title('File not found')
            axs[idx].plot([])  # Add an empty plot to keep the subplot layout consistent

    # Hide any empty subplots
    for idx in range(len(param_grid), len(axs)):
        axs[idx].set_visible(False)

    for ax in axs:
        ax.set_xlabel('Episode')
        ax.set_ylabel('Total Reward')
        ax.grid(True)

    plt.tight_layout()
    plt.show()

# Define the parameter grid
param_grid = list(product(
    [1e-4, 3e-4, 1e-3],  # Learning rates
    [0.1, 0.2, 0.3],     # Epsilon values
    [5, 10, 20],         # Epochs
    [32, 64, 128]        # Batch sizes
))

# Plot the rewards
plot_rewards(param_grid)


In [ ]:
import matplotlib.pyplot as plt
import pickle
import os
from itertools import product

def plot_rewards_2(param_grid, file_template='rewards_lr_{lr}_epsilon_{epsilon}_k_epochs_{k_epochs}_batch_size_{batch_size}.pkl', interval=10000):
    num_params = len(param_grid)
    num_cols = 4
    num_rows = (num_params + num_cols - 1) // num_cols  # Calculate rows needed

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 5 * num_rows), sharex=True, sharey=True)
    axs = axs.flatten()

    for idx, params in enumerate(param_grid):
        lr, epsilon, k_epochs, batch_size = params
        file_name = "./rewards/" + file_template.format(lr=lr, epsilon=epsilon, k_epochs=k_epochs, batch_size=batch_size)
        
        if os.path.exists(file_name):
            with open(file_name, 'rb') as f:
                rewards = pickle.load(f)
            
            # Calculate average rewards over intervals
            avg_rewards = [sum(rewards[i:i + interval]) / interval for i in range(0, len(rewards), interval)]
            
            label = f'LR={lr}, ε={epsilon}, Epochs={k_epochs}, Batch={batch_size}'
            axs[idx].plot(range(0, len(rewards), interval), avg_rewards, label=label)
            axs[idx].set_title(label)
            axs[idx].legend(loc='best')
        else:
            axs[idx].set_title('File not found')
            axs[idx].plot([])  # Add an empty plot to keep the subplot layout consistent

    # Hide any empty subplots
    for idx in range(len(param_grid), len(axs)):
        axs[idx].set_visible(False)

    for ax in axs:
        ax.set_xlabel('Episode')
        ax.set_ylabel('Total Reward')
        ax.grid(True)

    plt.tight_layout()
    plt.show()

# Define the parameter grid
param_grid = list(product(
    [1e-4, 3e-4, 1e-3],  # Learning rates
    [0.1, 0.2, 0.3],     # Epsilon values
    [5, 10, 20],         # Epochs
    [32, 64, 128]        # Batch sizes
))

# Plot the rewards
plot_rewards_2(param_grid)
